In [1]:
import numpy as np
import cv2


def rot(points, r):
    rx, ry, rz = np.deg2rad(r)
    rotx = np.array([
        [1., 0., 0.],
        [0., np.cos(rx), -np.sin(rx)],
        [0., np.sin(rx), np.cos(rx)],
    ])
    roty = np.array([
        [np.cos(ry), 0., -np.sin(ry)],
        [0., 1., 0.],
        [np.sin(ry), 0., np.cos(ry)],
    ])
    rotz = np.array([
        [np.cos(rz), -np.sin(rz), 0.],
        [np.sin(rz), np.cos(rz), 0.],
        [0., 0., 1.],
    ])
    return np.dot(rotz, np.dot(roty, np.dot(rotx, points.T))).T


class Pose():
    def __init__(self, p=np.zeros(3), r=np.zeros(3)):
        self.p = p.astype(np.float32)  # x, y, z
        self.r = r.astype(np.float32)  # x, y, z


class Entity():
    def __init__(self, pose):
        self.pose = pose  # 親に対しての位置姿勢
        self.children = []

    # set_hogeを一括でしたい?
    # pointを扱うクラス/faceを扱うクラスを継承しているべき
    def set_points(self, points):
        self.points = rot(points, self.pose.r)

    def set_faces(self, faces):
        self.faces = faces

    def view(self, r=np.zeros(3)):  # viewerに任せるべき?
        _points = rot(self.points, r)
        _points = _points - _points.min(axis=0)

        p = rot(self.pose.p, r)
        s = np.ceil(_points.max(0)).astype(np.int)
        s = np.insert(s[[0,2]], 2, [3])  # x,z
        frame = np.zeros(s).astype(np.uint8) + 64  ###
        frame = self._draw(frame, _points)
        p = (p[[0, 2]] - s[:2]/2).astype(np.int)
        return frame, p, s  # _pは親frameの原点中心
    
    def _draw(self, frame, points):
        for i, p in enumerate(points.astype(np.int)):
            frame = cv2.circle(frame, (p[2], p[0]), 5, (i*30, 0, 255-i*30), thickness=-1)
        return frame


class World():  # will be deprecated
    def __init__(self):
        self.children = []

    def add_children(self, children):  # 一括で突っ込めるようにする。
        self.children.extend(children)

    def view(self, r=np.zeros(3)):  # 視点クエリで視覚情報を返す
        s = np.array([1000, 500, 3])  # (x,z)平面 # TODO: 動的に決める
        frame = np.zeros(s, dtype=np.uint8)
        for child in world.children:
            f, _p, _s = child.view(r)  # (x,z), (x,z)
            _p += (s[:2] / 2.).astype(np.int)
            f = cv2.add(frame[_p[0]:_p[0]+_s[0],_p[1]:_p[1]+_s[1],:], f)
            frame[_p[0]:_p[0]+_s[0],_p[1]:_p[1]+_s[1],:] = f
        return frame


class Viewer():
    def __init__(self, world):
        self.world = world

    def view(self, r=np.zeros(3)):
        frame = self.world.view(r)
        w, h, _ = frame.shape
        cw, ch = int(w/2), int(h/2) # center
        frame = cv2.line(frame, (0, cw), (h, cw), color=(255, 255, 255))
        frame = cv2.line(frame, (ch, 0), (ch, w), color=(255, 255, 255))
        frame = np.transpose(frame, (1,0,2))[::-1]
        cv2.imshow("frame", frame)
        cv2.waitKey(10)


world = World()

for i in range(8):
    cube_pose = Pose(np.array([np.random.randint(-100, 100), 
                               np.random.randint(-100, 100), 
                               np.random.randint(-100, 100)]), 
                     np.array([np.random.randint(0, 360), 
                               np.random.randint(0, 360), 
                               np.random.randint(0, 360)]))
    cube_points = np.array([[-10, -10, -10],
                            [10, -10, -10],
                            [10, 10, -10],
                            [-10, 10, -10],
                            [-10, -10, 10],
                            [10, -10, 10],
                            [10, 10, 10],
                            [-10, 10, 10],
                           ], dtype=np.float32) * (np.random.random() + 0.25) * 2
    cube = Entity(pose=cube_pose)
    cube.set_points(points=cube_points)
    world.add_children([cube])

viewer = Viewer(world)
viewer.view(r=np.array([0.,0.,45.]))

for t in range(9000):
    viewer.view(r=np.array([np.float32(t),np.float32(t),np.float32(t)]))

In [ ]:
# 別にpointを直接使う必要はないはず？
# この階層のpointを便宜上使うってことにすればいい。
# もっと細かくしたいときは上の階層をこのpointsを使って定義してね
cube_faces = np.array([[0, 1, 2, 3],
                       [0, 1, 5, 4],
                       [0, 3, 7, 4],
                       [1, 2, 6, 5],
                       [3, 2, 6, 7],
                       [4, 5, 6, 7]])